In [1]:
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE, SVMSMOTE
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import r2_score,accuracy_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold, GridSearchCV

In [2]:
ls

 application_test.csv             HomeCredit_columns_description.csv
 application_train.csv            LightGBM.ipynb
 bureau_balance.csv              'Model Performance.ipynb'
 bureau.csv                       POS_CASH_balance.csv
 credit_card_balance.csv          previous_application.csv
 Data_Exploration_FullSet.ipynb   sample_submission.csv
 Data_Exploration.ipynb           submission.csv
 feat_eng.csv                     test_feat_eng.csv
 Feature_Eng.ipynb                train_feat_eng.csv
 full.csv


In [3]:
dataset = pd.read_csv("train_feat_eng.csv")

In [4]:
dataset.shape

(307511, 125)

In [5]:
targetSet = dataset['TARGET']
targetSet = pd.DataFrame({'TARGET': targetSet})
targetSet

,TARGET
0,1
1,0
2,0
3,0
4,0
...,...
307506,0
307507,0
307508,0
307509,1


In [6]:
dataset = dataset.drop('TARGET', 1)

In [7]:
dataset.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_CREDIT_LIMIT_ACTUAL',
       'AMT_CREDIT_TO_ANNUITY_RATIO'],
      dtype='object', length=124)

In [8]:
testset = pd.read_csv("test_feat_eng.csv")

In [9]:
testset.shape

(48744, 124)

In [10]:
testset.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_TO_ANNUITY_RATIO
0,0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,27.664697
1,1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,...,0,0,0.0,0.0,0.0,0.0,0.0,3.0,NaN,12.824870
2,2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,...,0,0,0.0,0.0,0.0,0.0,1.0,4.0,131718.75,9.505482
3,3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,...,0,0,0.0,0.0,0.0,0.0,0.0,3.0,225000.00,32.130726
4,4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.506034


In [11]:
testset[testset.isna()].count()

Unnamed: 0                     0
SK_ID_CURR                     0
NAME_CONTRACT_TYPE             0
CODE_GENDER                    0
FLAG_OWN_CAR                   0
                              ..
AMT_REQ_CREDIT_BUREAU_MON      0
AMT_REQ_CREDIT_BUREAU_QRT      0
AMT_REQ_CREDIT_BUREAU_YEAR     0
AMT_CREDIT_LIMIT_ACTUAL        0
AMT_CREDIT_TO_ANNUITY_RATIO    0
Length: 124, dtype: int64

In [12]:
dataset.head()

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AMT_CREDIT_LIMIT_ACTUAL,AMT_CREDIT_TO_ANNUITY_RATIO
0,0,100002,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,16.461104
1,1,100003,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36.234085
2,2,100004,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.000000
3,3,100006,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,270000.0,10.532818
4,4,100007,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.461618


In [13]:
dataset.shape

(307511, 124)

In [14]:
testset.shape

(48744, 124)

In [15]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Get  columns whose data type is object i.e. string
filteredColumns = dataset.dtypes[dataset.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

# Label Encoding if there are two categorical labels on a feature)
for col in listOfColumnNames:
    if dataset.dtypes[col] == np.object:
        # If 2 or fewer unique categories
        if len(list(dataset[col].unique())) <= 2:
            # Train on the training data
            le.fit(dataset[col])
            le_count += 1
            
print('%d columns were label encoded.' % le_count)


# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Get  columns whose data type is object i.e. string
filteredColumns = testset.dtypes[testset.dtypes == np.object]
# list of columns whose data type is object i.e. string
listOfColumnNames = list(filteredColumns.index)
print(listOfColumnNames)

# Label Encoding if there are two categorical labels on a feature)
for col in listOfColumnNames:
    if testset.dtypes[col] == np.object:
        # If 2 or fewer unique categories
        if len(list(testset[col].unique())) <= 2:
            # Train on the training data
            le.fit(testset[col])
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

# One-Hot Encoding (> 2 categorical labels for a feature)
num_dataset = pd.get_dummies(dataset,drop_first=True)

num_testset = pd.get_dummies(testset,drop_first=True)

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
3 columns were label encoded.
['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE', 'FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']
4 columns were label encoded.


In [16]:
num_dataset.shape

(307511, 232)

In [17]:
num_dataset.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       ...
       'FONDKAPREMONT_MODE_reg oper spec account',
       'HOUSETYPE_MODE_specific housing', 'HOUSETYPE_MODE_terraced house',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=232)

In [18]:
num_testset.shape

(48744, 229)

In [19]:
index = num_dataset.columns.isin(num_testset.columns)
~index

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False,

In [20]:
#remove columns from train set that are not in the test set, This is need to make a submission csv
num_dataset = num_dataset.drop(num_dataset.columns[~index], 1)

In [21]:
null_rows = num_dataset[num_dataset.isnull().any(axis=1)]
null_rows

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,...,0,0,0,0,0,0,0,1,0,0
1,1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,...,0,0,0,0,0,0,0,0,0,0
2,2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,...,0,0,0,0,0,0,0,0,0,0
3,3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,...,0,0,0,0,0,0,0,0,0,0
4,4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,307506,456251,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,...,0,0,0,0,0,0,0,1,0,0
307507,307507,456252,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,...,0,0,0,0,0,0,0,1,0,0
307508,307508,456253,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,...,0,0,0,0,0,0,1,0,0,0
307509,307509,456254,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,...,0,0,0,0,0,0,0,1,0,0


In [22]:
null_rows.shape

(296160, 229)

What percent of the rows in the data have a null value in at least one column?

In [23]:
null_rows.shape[0]/dataset.shape[0]

0.9630874993089678

97% of the rows have at least one column with a null value. 
How will we handle null values?

In [24]:
num_dataset.shape

(307511, 229)

In [25]:
num_testset.shape

(48744, 229)

In [26]:
dataset.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER',
       'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY',
       ...
       'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 'AMT_REQ_CREDIT_BUREAU_HOUR',
       'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
       'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT',
       'AMT_REQ_CREDIT_BUREAU_YEAR', 'AMT_CREDIT_LIMIT_ACTUAL',
       'AMT_CREDIT_TO_ANNUITY_RATIO'],
      dtype='object', length=124)

In [27]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(num_dataset)
new_num_dataset = pd.DataFrame(imp.transform(num_dataset))
new_num_dataset.columns = num_dataset.columns
new_num_dataset.index = num_dataset.index
num_dataset = new_num_dataset

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(num_testset)
new_num_testset = pd.DataFrame(imp.transform(num_testset))
new_num_testset.columns = num_testset.columns
new_num_testset.index = num_testset.index
num_testset = new_num_testset

null_rows = num_dataset[num_dataset.isnull().any(axis=1)]
null_rows

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes


In [28]:
num_dataset.shape

(307511, 229)

In [29]:
num_testset.shape

(48744, 229)

In [30]:
num_dataset.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,0.0,100002.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1.0,100003.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,100004.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,100006.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,100007.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
targetSet['TARGET']

0         1
1         0
2         0
3         0
4         0
         ..
307506    0
307507    0
307508    0
307509    1
307510    0
Name: TARGET, Length: 307511, dtype: int64

In [32]:
correls = {}
counter = -1
for col in num_dataset.columns:
    counter+=1
    c = targetSet['TARGET'].corr(num_dataset[col])
    if c is not None:
        correls[col] = float(c)
    else:
        correls[col] = (None)

In [33]:
for w in sorted(correls, key=correls.get, reverse=True):
    print(w, correls[w])

DAYS_BIRTH 0.07823930830982712
REGION_RATING_CLIENT_W_CITY 0.0608926675648237
REGION_RATING_CLIENT 0.05889901494571248
NAME_INCOME_TYPE_Working 0.05748118481836544
DAYS_LAST_PHONE_CHANGE 0.05521847562884968
CODE_GENDER_M 0.05471268197127142
DAYS_ID_PUBLISH 0.051457172607057504
REG_CITY_NOT_WORK_CITY 0.05099446436812682
NAME_EDUCATION_TYPE_Secondary / secondary special 0.04982373084813319
FLAG_EMP_PHONE 0.045982219716592596
REG_CITY_NOT_LIVE_CITY 0.044395374805701
FLAG_DOCUMENT_3 0.04434634685114509
OCCUPATION_TYPE_Laborers 0.04301895495530795
DAYS_REGISTRATION 0.0419748628314172
LIVE_CITY_NOT_WORK_CITY 0.032518341101499294
DEF_30_CNT_SOCIAL_CIRCLE 0.03222153390591384
DEF_60_CNT_SOCIAL_CIRCLE 0.03125121111173648
OCCUPATION_TYPE_Drivers 0.03030349991494772
NAME_HOUSING_TYPE_With parents 0.02996585666593053
ORGANIZATION_TYPE_Self-employed 0.029138816170825095
FLAG_WORK_PHONE 0.028524322363217617
OCCUPATION_TYPE_Low-skill Laborers 0.027590818570870146
NAME_FAMILY_STATUS_Single / not marrie

In [34]:
X_train, X_test, y_train, y_test = train_test_split(num_dataset, 
                                    targetSet['TARGET'], test_size = .2, random_state=42,shuffle=True)

In [35]:
# SMOTE: Overclassing Minority Classification

sum_1 = sum(y_train==1)
sum_0 = sum(y_train==0)

print('Total Data', len(y_train))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

print("\nPerforming SMOTE on training data set\n")
sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

sum_1 = sum(y_train_res==1)
sum_0 = sum(y_train_res==0)

print('Total Data', len(y_train_res))
print('Imbalance: [-]', sum_0, '[+]', sum_1)

Total Data 246008
Imbalance: [-] 226132 [+] 19876

Performing SMOTE on training data set

Total Data 452264
Imbalance: [-] 226132 [+] 226132


In [36]:
#amount of rows in input data
X_train_res.shape[0]

452264

In [37]:
#amount of columns in input data
len(X_train_res.columns)

229

In [38]:
X_train_res['AMT_CREDIT_LIMIT_ACTUAL']

0         151875.000000
1              0.000000
2              0.000000
3              0.000000
4          93145.161290
              ...      
452259         0.000000
452260     90000.000000
452261     25379.000232
452262         0.000000
452263         0.000000
Name: AMT_CREDIT_LIMIT_ACTUAL, Length: 452264, dtype: float64

In [39]:
model = DecisionTreeClassifier()
model.fit(X_train_res, y_train_res)
y_pred=model.predict(X_test)

In [40]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8467879615628506


In [41]:
print(confusion_matrix(y_test,y_pred))

[[51244  5310]
 [ 4113   836]]


In [42]:
print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

Decision Tree TRAIN score with  1.0
Decision Tree TEST score with  0.8467879615628506


In [43]:
auc = roc_auc_score(y_test, y_pred)
auc

0.5375152259450897

In [44]:
num_dataset.to_csv("full.csv")

In [45]:
#cross_validate
cv_results = cross_validate(model, X_train_res, y_train_res, cv=10)

In [46]:
#score with cross validation
np.mean(cv_results['test_score'])

0.9071542825780942

In [47]:
param_grid = {"criterion": ['entropy', 'gini'],
"min_samples_split": [5,10,15],
"max_depth": [2,3,5],
"min_samples_leaf": [5,10,15],
"max_leaf_nodes": [5,10,15],
}
CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
CV_rfc.fit(X_train_res, y_train_res)
CV_rfc.best_params_

{'criterion': 'gini',
 'max_depth': 5,
 'max_leaf_nodes': 15,
 'min_samples_leaf': 5,
 'min_samples_split': 5}

In [48]:
num_testset.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       ...
       'FONDKAPREMONT_MODE_reg oper spec account',
       'HOUSETYPE_MODE_specific housing', 'HOUSETYPE_MODE_terraced house',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=229)

In [49]:
model = DecisionTreeClassifier(**CV_rfc.best_params_)
model.fit(X_train_res, y_train_res)
pred=model.predict_proba(num_testset)
pred

array([[0.83342366, 0.16657634],
       [0.29173641, 0.70826359],
       [0.82855871, 0.17144129],
       ...,
       [0.56933469, 0.43066531],
       [0.82855871, 0.17144129],
       [0.56933469, 0.43066531]])

##### pred[:, 1]

In [50]:
probDF = pd.DataFrame(pred[:, 1], columns =['Probs']) 
probDF[probDF.isna()==True]

,Probs
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
48739,NaN
48740,NaN
48741,NaN
48742,NaN


In [51]:
num_testset.columns

Index(['Unnamed: 0', 'SK_ID_CURR', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL',
       'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE',
       'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED',
       ...
       'FONDKAPREMONT_MODE_reg oper spec account',
       'HOUSETYPE_MODE_specific housing', 'HOUSETYPE_MODE_terraced house',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden',
       'EMERGENCYSTATE_MODE_Yes'],
      dtype='object', length=229)

In [61]:
IDs = num_testset[['SK_ID_CURR']].astype('int64', copy=False)
IDs

,SK_ID_CURR
0,100001
1,100005
2,100013
3,100028
4,100038
...,...
48739,456221
48740,456222
48741,456223
48742,456224


In [62]:
submission = pd.DataFrame({'SK_ID_CURR': IDs.SK_ID_CURR.astype('int64', copy=False).tolist(),
'TARGET': probDF.Probs.tolist()})

In [63]:
submission.head()

,SK_ID_CURR,TARGET
0,100001,0.166576
1,100005,0.708264
2,100013,0.171441
3,100028,0.228779
4,100038,0.988248


In [64]:
submission[~submission.isna()]

,SK_ID_CURR,TARGET
0,100001,0.166576
1,100005,0.708264
2,100013,0.171441
3,100028,0.228779
4,100038,0.988248
...,...,...
48739,456221,0.125978
48740,456222,0.988248
48741,456223,0.430665
48742,456224,0.171441


In [65]:
submission.to_csv("submission.csv", index=False)

In [57]:
#print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [58]:
#print("Decision Tree TRAIN score with ",format(model.score(X_train_res, y_train_res)))
#print("Decision Tree TEST score with ",format(model.score(X_test, y_test)))

In [59]:
#print(confusion_matrix(y_test,y_pred))

In [60]:
#auc = roc_auc_score(y_test, y_pred)
#auc